In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
from IPython import display
from base64 import b64decode

# 5 Solving nonlinear equations
When it comes to solving equations, there are only handful of types for which we can actually work on paper to obtain the analytical solution. Using numerical methods to obtain a reasonable approximations to the solutions thus is of great importance. Notice the following equations in one variable which are extremely hard or impossible to solve analytically.
* $e^x=x^2$
* $\sin(x^2)=x^2+2x+3$
* $\sin^3(x)+2\sin^2(x)+\cos(x)=5e^{\cos(x)}$

In multiple variables, it only gets worse. In a later chapter on numerical integration, we will want a solution $(x_1,x_2,w_1,w_2)$ to the system
$$\begin{align*}
w_1+w_2 &=2,\\
w_1x_1 +w_2x_2 &=0,\\
w_1x_1^2+w_2x_2^2 &= \frac{2}{3},\\
w_1x_1^3+w_2x_2^3 &=0.
\end{align*}$$

In [ ]:
def function(x):
    return np.cos(x) - np.exp(x)

def functionDerivative(x):
    return (-np.sin(x)-np.exp(x))

In [ ]:
# Method of bisection
def bisect(a,b,func,tol):
    fa = func(a)
    fb = func(b)

    count=0
    data = np.array([count,a,b,fa,fb, b-a])

    while b-a>tol:
        x=(a+b)/2
        fx=func(x)
        count+=1
        if fa*fx<=0:
            b=x
            fb=fx
        if fx*fb<0:
            a=x
            fa=fx
        datanew = np.array([count, a, b, fa, fb, b-a])
        data = np.vstack((data,datanew))
    return x, data

In [ ]:
x, data = bisect(-2,-0.5,function,10**(-6))
print(tabulate(data, headers=['#', 'a', 'b', 'f(a)', 'f(b)', 'b-a'],
               tablefmt='orgtbl'))
print("Solution : ",x)

|   # |        a |        b |         f(a) |        f(b) |         b-a |
|-----+----------+----------+--------------+-------------+-------------|
|   0 | -2       | -0.5     | -0.551482    | 0.271052    | 1.5         |
|   1 | -2       | -1.25    | -0.551482    | 0.0288176   | 0.75        |
|   2 | -1.625   | -1.25    | -0.251089    | 0.0288176   | 0.375       |
|   3 | -1.4375  | -1.25    | -0.104619    | 0.0288176   | 0.1875      |
|   4 | -1.34375 | -1.25    | -0.0357649   | 0.0288176   | 0.09375     |
|   5 | -1.29688 | -1.25    | -0.00287615  | 0.0288176   | 0.046875    |
|   6 | -1.29688 | -1.27344 | -0.00287615  | 0.013128    | 0.0234375   |
|   7 | -1.29688 | -1.28516 | -0.00287615  | 0.00516429  | 0.0117188   |
|   8 | -1.29688 | -1.29102 | -0.00287615  | 0.00115353  | 0.00585938  |
|   9 | -1.29395 | -1.29102 | -0.00085896  | 0.00115353  | 0.00292969  |
|  10 | -1.29395 | -1.29248 | -0.00085896  | 0.000147875 | 0.00146484  |
|  11 | -1.29321 | -1.29248 | -0.000355395 | 0.0001

# Newton's Method

In [ ]:
#Newton's method
def newtons(x0,function,functionDerivative,tol):
    error = 1
    count=0
    data = np.array([0,x0])
    while error>tol:
        count+=1
        xnew = x0 - function(x0)/functionDerivative(x0)
        error = np.abs(xnew-x0)
        x0=xnew
        datanew = np.array([count, xnew])
        data = np.vstack((data,datanew))
    return xnew,data

In [ ]:
def function(x):
    return np.exp(x)-x**2
def functionDerivative(x):
    return np.exp(x)-2*x

x,data = newtons(-1,function, functionDerivative, 10**(-14))
for i in range(len(data[:,1])):
    print(data[i,1])
print("Solution = {:.14f}".format(x))

-1.0
-0.7330436052454454
-0.703807786324133
-0.7034674683317975
-0.7034674224983924
-0.7034674224983917
Solution = -0.70346742249839


# Secant Method

In [ ]:
# Secant Method
def secant(x0,x1,function,tol):
    error = 1
    count=0
    data = np.array([0,x0,x1])
    while error>tol:
        count+=1
        xnew = x1 - function(x1)/((function(x1)-function(x0))/(x1-x0))
        error = np.abs(xnew-x1)
        x0=x1
        x1=xnew
        datanew = np.array([count, x0,x1])
        data = np.vstack((data,datanew))
    return xnew,data

In [ ]:
x, data = secant(-2,-0.5,function,10**(-6))
print(tabulate(data, headers=['#', 'x0', 'x1'], tablefmt='orgtbl'))
print("Solution : ",x)

|   # |        x0 |        x1 |
|-----+-----------+-----------|
|   0 | -2        | -0.5      |
|   1 | -0.5      | -0.626693 |
|   2 | -0.626693 | -0.710172 |
|   3 | -0.710172 | -0.70326  |
|   4 | -0.70326  | -0.703467 |
|   5 | -0.703467 | -0.703467 |
Solution :  -0.7034674225436495


# Equation solving in higher dimension

In [ ]:
def function2D(x):
    return np.array([x[0]**3-x[1],x[0]+np.sin(x[1])+3])

def function2DJacobian(x):
    return np.array([[3*x[0]**2,-1],[1,np.cos(x[1])]])

In [ ]:
def newtons2D(x0,func,funcJac,tol):
    error = 1
    count=0
    data = np.array([count, x0[0], x0[1], error])
    while error>tol and count < 1000:
        count+=1
        xnew = x0 - np.dot(np.linalg.inv(funcJac(x0)), func(x0))
        error = np.linalg.norm(xnew-x0)
        x0=xnew
        datanew = np.array([count, xnew[0], xnew[1], error])
        data = np.vstack((data,datanew))
    return xnew,data

In [ ]:
x0 = np.array([-2, -15])
x,data = newtons2D(x0,function2D, function2DJacobian, 10**(-8))
print(tabulate(data, headers=['#', 'x', 'y','error'], tablefmt='orgtbl'))
print("Solution = ", x)
print("Function = ", function2D(x))

|   # |        x |        y |      error |
|-----+----------+----------+------------|
|   0 | -2       | -15      | 1          |
|   1 | -2.61212 | -15.3454 | 0.70285    |
|   2 | -2.48258 | -15.1713 | 0.216983   |
|   3 | -2.4747  | -15.155  | 0.0181575  |
|   4 | -2.47467 | -15.1549 | 0.00012237 |
|   5 | -2.47467 | -15.1549 | 5.16e-09   |
Solution =  [ -2.47467012 -15.15486052]
Function =  [-5.32907052e-15 -8.88178420e-16]


In [ ]:
# N-dimensional Newton
def newtonsHiDim(x0,func,funcJac,tol):
    error = 1
    count=0
    while error>tol and count < 1000:
        count+=1
        print(x0)
        xnew = x0 - np.dot(np.linalg.inv(funcJac(x0)), func(x0))
        error = np.linalg.norm(xnew-x0)
        x0=xnew
    return xnew

In [ ]:
def function(x):
    # c1=x[0], c2=x[1], x1=x[2], x2=x[3]
    return np.array([x[0] + x[1]-2,
                     x[0]*x[2] + x[1]*x[3],
                     x[0]*(x[2]**2) + x[1]*(x[3]**2)-2/3,
                     x[0]*(x[2]**3) + x[1]*(x[3]**3)
                    ])

def functionJacobian(x):
    return np.array([[1, 1, 0, 0],
                    [x[2] ,x[3] ,x[0] ,x[1]],
                    [x[2]**2 ,x[3]**2 ,2*x[0]*x[2] ,2*x[1]*x[3]],
                    [x[2]**3 ,x[3]**3 ,3*x[0]*x[2]**2 ,3*x[1]*x[3]**2]
                    ])

In [ ]:
x0 = np.array([1,1,-1,1])
x = newtonsHiDim(x0,function, functionJacobian, 10**(-12))
print("Solution = ", x)
print("Function = ", function(x))

[ 1  1 -1  1]
[ 1.          1.         -0.66666667  0.66666667]
[ 1.          1.         -0.58333333  0.58333333]
[ 1.          1.         -0.57738095  0.57738095]
[ 1.          1.         -0.57735027  0.57735027]
[ 1.          1.         -0.57735027  0.57735027]
Solution =  [ 1.          1.         -0.57735027  0.57735027]
Function =  [0. 0. 0. 0.]


# Fixed Point Iteration